#Python_Programming

---

---

I have explained the concepts of **Projections**, **Coordinate reference system**, **Transformation**, **NDVI**, and **NDWI** verbaly.

Also I have done coding to, 

*   claculate Indices such as NDVI, NDWI and NDBI.
*   create composite images such as FCC, NaturalColor.
*   extract the region of intrest out of the result.




##Map Projections
---
Earth is not a Plane surface but the maps we see are printed on 2D paper. Have you wondered how this can be achieved? The answer is, earth surface is projected to a plane surface and it is scaled down to small Paper size(like A0). 

Now, lets us discuss different types of projections. But why do we need different types of projections? Our Earth is not smooth Spherical in shape as we see in movies. When only the terrain is mapped earth looks like a crushed paper ball [Earth IMG](https://images.app.goo.gl/8HiHxPMbW8p1obg89). 

So it is obvious that a single projection cannot be used across the globe to map all the features. Projection are chosen by considering which property of the geographical features to be conserved such as *area, shape, distance, or direction*. All four properties cannot be conserved in one projection, so the projection has to be chosen based on what property is more important to be conserved. Also, the surface onto which the features are projected plays an important role. Diffrent types of surfaces are Conical, Cylindrical, Planar.


*   Counties near the polar regions  (Greenland, New Zealand) follow **Conical projection**.
*   Countries near Equator (Brazil) follow **Cylindrical projection**.
*   Poles are projected by **Planar/Azimuthal projection**.

[Projection IMG](https://images.app.goo.gl/MmBMubQMyhw7EGNB9).

To further understand this concept lets learn about **coordinate reference system**

##Coordinate Reference System

---
*A system used to register and measure horizontal and vertical distance on a map ~ C.P. Lo*

It is a coordinate-based local, regional, or global system used to locate geographical features.  Also, it defines a specific map projection, as well as transformations between different reference systems

WGS 84 is a coordinate reference system that is developed by the U.S. Department of Defense. It can also be said as a "global reference system" for geospatial information and is
the reference system for the Global Positioning System (GPS). So the best fit of the entire globe is WGS 84.  But to get more accurate information about any location we have to choose the projection as discussed earlier. 

Data collected for a project can be from different sources hence they all may not be on the same projection, it is very important to only overlay and compare maps of the same projection, or to calculate distance or area in metric units from decimal coordinate system projection WGS 84 transformation is used.  This leads to the concept of **Transformation**

##Transformation. 

---

Changing from one coordinate reference system to another is called Transformation. While opening a map in any GIS software such as (ArcMap) the first map loaded in a project will be taken as the reference coordinate system and the other maps loaded will be automatically projected to previous layer projection. Hence it is very important to check the projection of all maps and project it to the same coordinate system before working on them. 
Now what to do if the cell size doesn't be same on diffrent Raster ? We have to do **Resampling**

##Resampling

---

The process of deriving pixel values for a new image from an existing image is called as resampling. For Example in NDBI, We are about to compute with Bands SWIR1375, NIR which of resolution 20m and 10m respectively. Hence SWIR1375 is Resampled to match the NIR resolution using **nearest neighbor** 

###Nearest Neighbor

---
Nearest Neighbor Algotithm takes the cell center from the input raster data set to determine the closest cell center of the output raster.

Thus we have discussed briefly Map Projection, Coordinate Reference System, Resampling, Nearest Neighbor and why Transformation is important.

Next lets us discuss about Calculation of Indices.

##INDICES


###Normalized Difference Vegetation Index (NDVI)
---
NDVI is the most commonly used vegetation index for monitoring the condition of vegetation/health of plantation.

This is calculated using Satellite imagery (Sentinal, LandSat, Cartosat.. ) which is capable of capturing information in multispectral bands. 

*Formula:* $NDVI = \frac{\left(NIR - Red\right)} {\left(NIR + Red\right)}$

Red band and NIR band are used to calculate the index. The calculated value ranges from -1 to 1. Also below is the value range for different types of vegetation.


*   less than 0.1 - Barren Rocks, Sand, or Snow
*   0.2 to 0.3    - shrub and grassland
*   0.6 to 0.8    - tropical rainforests

[ESA (reference for index)](https://earthobservatory.nasa.gov/features/MeasuringVegetation)

*Explanation:* Red, Green, Blue are the primary colors, and combinations of these colors only produce other colors seen by the human eye. Plants look green to our eyes because it observes most of the Red and Blue light when sun rays fall on it. Through experiments, it is found that chlorophyll content is observing more Red light so we can conclude that if emitted Red light is low from any region it contains high chlorophyll and vice-versa. Also, NIR would be strong for vegetation too. 

[NDVI Image](https://images.app.goo.gl/HJ7FM8yYZgcWokmSA)

Hence by calculating this Index we can comment on the vegetation in a particular region. 



###Normalized Difference Water Index (NDWI)

---
NDWI is the most commonly used water index for monitoring the condition of 


*   Water content in Plants (Vegetation)
*   Water content in water bodies (Pond, Lakes, etc.,)

This is claculated using Satellite imagery (Sentinal, LandSat, Cartosat.. ) which is capable of capturing information in multispectral bands.

*Formula:*
> Water content in Plants

$NDWI_{Plants} = \frac{\left(NIR - SWIR\right)} {\left(NIR + SWIR\right)}$

> Water content in water bodies

 $NDWI_{WaterBodies} = \frac{\left(Green - NIR\right)} {\left(Green + NIR\right)}$.

*Explanation:*It varies from -1 to +1, it depends on the type of vegetation and cover. The high NDWI values correspond to high plant water content and coating of high plant fraction. Low NDWI values ​ correspond to low vegetation content and cover with low vegetation. During periods of water stress the NDWI rate will decrease.

The NDWI index for assessing risk of fire is used to determine the presence of moisture in vegetation cover. Higher NDWI values ​​indicate sufficient moisture, while a low value indicates water stress












#Imports

In [1]:
#Basic Library
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import os 
import glob
from termcolor import colored

#Spatial Library
import fiona
import geopandas as gpd
import rasterio
from rasterio import plot
from rasterio import mask
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject

#Satellite Imgag

In [2]:
def read_data():
    '''Enter the number to read the data from Sentinal 2 Imagery...'''
    path = input("Enter the path to folder for satellit: \n")
    pattern = input("\nEnter the pattern of satellite with * \n like (*.tif or *.jp2): ")
    p = os.path.join(path, pattern)
    file = glob.glob(p)
    band = ['Band1','Band2','Band3','Band4','Band5','Band6','Band7','Band8','Band9','Band10','Band11','Band12','Band13']
    spectral = ["CoastalAerosol", "Blue", "Green", "Red", "VNIR705", "VNIR740", "VNIR783", "VNIR842", "VNIR865",
                      "SWIR940", "SWIR1375", "SWIR1610", "SWIR2019"]
    spectral_bands = zip(band, spectral)
    print("\nSentinal Reference:")
    for s in spectral_bands:
        print(s)
    
    ca = rasterio.open(file[0])
    b = rasterio.open(file[1])
    g = rasterio.open(file[2])
    r = rasterio.open(file[3])
    v1 = rasterio.open(file[4])
    v2 = rasterio.open(file[5])
    v3 = rasterio.open(file[6])
    nir = rasterio.open(file[7])
    v4 = rasterio.open(file[8])
    wv = rasterio.open(file[9])
    s1 = rasterio.open(file[10])
    s2 = rasterio.open(file[11])
    s3 = rasterio.open(file[12])
    
    return ca, b, g, r, v1, v2, v3, nir, v4, wv, s1, s2, s3 

##Band Combinatins

In [3]:
def band_combination(b1, b2, b3, name):
    '''
    Enter the Bands information by refering the example below. 
    Eg.,
        Natural Colour (Red, Green, Blue, "TCC")
        False Colour (Green, Blue, nir, "FCC")'''
    
    path = input("Enter the path to to save the band combination: ")
    comb = "{}.tif".format(name)
    
    p = os.path.join(path, comb)
    
    combinations = rasterio.open(p, 'w',
                        driver = "Gtiff",
                        width=b1.width,
                        height=b1.height,
                        crs=b1.crs,
                        count=3,
                        transform=b1.transform,
                        dtype=b1.dtypes[0])
    combinations.write(b3.read(1),3)
    combinations.write(b2.read(1),2)
    combinations.write(b1.read(1),1)
    combinations.close()
    print(colored("\nSuccess. {} image has been created at location {}".format(name, p), attrs=['bold']))

##Calculating Index

In [4]:
def creat_index(b1, b2, name):

    path = input("Enter the path to to save the band combination: ")
    comb = "{}.tif".format(name)
    p1 = os.path.join(path, comb)
        
    if b1.height != b2.height:
        p2 = Resample_Raster(b1, b2.height/b1.height, path, name)
        b1 = rasterio.open(p2)
        
    n = b1.read(1)
    r = b2.read(1)
    comp = (n - r)/(n + r)

    metadata = b2.meta
    metadata.update({'count' :1,'dtype':'float64','driver': 'GTiff'})
    result = rasterio.open(p1, "w", **metadata)
    result.write(comp,1)
    result.close()
    
    print(colored("\nSuccess. {} image has been created at location {}".format(name, p1), attrs=['bold']))

In [5]:
def Resample_Raster(dataset, upscale_factor, path, name):
    
    data = dataset.read(out_shape=(dataset.count,int(dataset.height * upscale_factor),int(dataset.width * upscale_factor)),
                        resampling=Resampling.bilinear)
    
    # scale image transform
    transform = dataset.transform * dataset.transform.scale((dataset.width / data.shape[-1]),(dataset.height / data.shape[-2]))

    metadata = dataset.meta
    metadata.update({'dtype':'uint16','driver': 'GTiff', 
                     'height':dataset.height * upscale_factor, 'width':dataset.width * upscale_factor})
    
    resamplee = "{}_Resample.tif".format(name)
    p3 = os.path.join(path,resamplee)
    result = rasterio.open(p3, "w", **metadata)
    result.write(data)
    result.close()
    
    return p3

#Getting Region of Interest (.shp)

In [14]:
def roi():
    
    path1 = input("Enter the file path of for ROI (.shp format): ")    
    with fiona.open(path1, "r") as shp:
        shapes = []
        vec_crs = shp.crs
        for feature in shp:
            shapes.append(feature["geometry"])

    path2 = input("Enter the file path of Raster Image: ") 
    print("\n\n")
    with rasterio.open(path2) as src:
        if src.crs != vec_crs:
            path3 = Project_Raster(path2, vec_crs)
        else:
            path3 = path2
            print("Projection Matches!")
    print("\n\n")
            
    with rasterio.open(path3) as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
        out_meta = src.meta
        x, y = src.res
        h = out_image.shape[1]
        w = out_image.shape[2]
    
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    
    path3 = input("Enter the file path of folder to save the ROI image: ")
    name = input("Enter the name with extension (.tif): ")
    p = os.path.join(path3, name)

    with rasterio.open(p, "w", **out_meta) as dest:
        dest.write(out_image)

    print(colored("\nSuccess. The ROI image {} has been stored at location {} ".format(name, path3), attrs=['bold']))   

In [7]:
def Project_Raster(path2, dst_crs):
    with rasterio.open(path2) as src:
        transform, width, height = calculate_default_transform(src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
    
        print("Raster CRS and Vector CRS Doesnt match. \nSo the Raster is Reprojected to Vector project: ")
        path3 = input("Enter the file path of folder and Name to save the Reprojected Raster: ")

        with rasterio.open(path3, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
    return path3

# Main Program

In [8]:
#Reading Data
#L1C_T43PHL_A017491_20200712T051957\S2B_MSIL1C_20200712T050659_N0209_R019_T43PHL_20200712T085717.SAFE\GRANULE\L1C_T43PHL_A017491_20200712T051957\IMG_DATA
CoastalAerosol, Blue, Green, Red, VNIR705, VNIR740, VNIR783, NIR, VNIR865, WaterVapour, SWIR1375, SWIR1610, SWIR2019  = read_data()

Enter the path to folder for satellit: 
L1C_T43PHL_A017491_20200712T051957\S2B_MSIL1C_20200712T050659_N0209_R019_T43PHL_20200712T085717.SAFE\GRANULE\L1C_T43PHL_A017491_20200712T051957\IMG_DATA

Enter the pattern of satellite with * 
 like (*.tif or *.jp2): *.jp2

Sentinal Reference:
('Band1', 'CoastalAerosol')
('Band2', 'Blue')
('Band3', 'Green')
('Band4', 'Red')
('Band5', 'VNIR705')
('Band6', 'VNIR740')
('Band7', 'VNIR783')
('Band8', 'VNIR842')
('Band9', 'VNIR865')
('Band10', 'SWIR940')
('Band11', 'SWIR1375')
('Band12', 'SWIR1610')
('Band13', 'SWIR2019')


In [9]:
#Band Combinations
#True Colour
band_combination(Red, Green, Blue, "TCC")

Enter the path to to save the band combination: Output\CompositeImages

Success. TCC image has been created at location Output\CompositeImages\TCC.tif


In [10]:
#False Colour
band_combination(Green, Blue, NIR, "FCC")

#Create your own composite image by passing 3 band name and Name to store the image
#band_combination(, , , " ")
#band_combination(, , , " ")

Enter the path to to save the band combination: Output\CompositeImages

Success. FCC image has been created at location Output\CompositeImages\FCC.tif


In [11]:
#Calculate NDVI
creat_index(NIR, Red, "NDVI")

Enter the path to to save the band combination: Output\IndexImages


C:\Users\User\anaconda3\envs\Python_foundation\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]



Success. NDVI image has been created at location Output\IndexImages\NDVI.tif


In [12]:
#Calculate NDBI
creat_index(SWIR1375, NIR, "NDBI")

#Calculate own index
#creat_index(, , " ")
#creat_index(, , " ")

Enter the path to to save the band combination: Output\IndexImages


C:\Users\User\anaconda3\envs\Python_foundation\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]



Success. NDBI image has been created at location Output\IndexImages\NDBI.tif


In [15]:
#Extract Region od Interest
roi()

Enter the file path of for ROI (.shp format): Madurai\Region_of_intrest.shp
Enter the file path of Raster Image: Output\IndexImages\NDBI.tif



Raster CRS and Vector CRS Doesnt match. 
So the Raster is Reprojected to Vector project: 
Enter the file path of folder and Name to save the Reprojected Raster: Output\IndexImages\Resampled_NDBI.tif



Enter the file path of folder to save the ROI image: Output\ROI
Enter the name with extension (.tif): Madurai_region.tif

Success. The ROI image Madurai_region.tif has been stored at location Output\ROI 
